# Numba - Just In Time Compilation for Python

Some technologies are so easy to use, and the effects are so powerful, that they appear to be magic.  Numba is one such technology.  

Let's start with Paul's prime sieve code from earlier.

In [ ]:
import  math
def sieve_primes(n):
    a = [True for x in range(n + 1)]
    i = 2
    while i <= math.sqrt(n):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
        i += 1
    return [i for i in range(2, len(a)) if a[i]]

In [ ]:
#Check it's working OK
sieve_primes(30)

In [ ]:
#Time it for all primes less than 5 million
N = 5000000
original_speed = %timeit -o sieve_primes(N)

And now with Numba

In [ ]:
from numba import jit

@jit
def numba_sieve_primes(n):
    a = [True for x in range(n + 1)]
    i = 2
    while i <= math.sqrt(n):
        if a[i]:
            for j in range(i*i, n + 1, i):
                a[j] = False
        i += 1
    return [i for i in range(2, len(a)) if a[i]]

In [ ]:
#Check it's working OK
numba_sieve_primes(30)

In [ ]:
#Time it 
numba_speed = %timeit -o numba_sieve_primes(N)

In [ ]:
print('Numba is {0:.1f} times faster than pure Python for the primes function'.format(original_speed.best/numba_speed.best))

**A significant speed up and all we did was type @jit**

### What is Numba doing?
#### Just in time compilation and compilation overhead

By adding the @jit decorator to our Python function, we are asking Numba to try to compile the function to machine code. The compilation happens the first time the function is called, hence the name **jit - just in time** compilation.  The compiler in question is the powerful open source compiler framework, LLVM.

This means that the first time a numba function is called there will be a delay due to the compilation but all subsequent calls will be fast.  Here's an explicit demonstration using `monte_carlo_pi`.

In [ ]:
from numba import jit
import random
import time

@jit
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
start = time.time()
pi = monte_carlo_pi(N)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

In [ ]:
start = time.time()
pi = monte_carlo_pi(N)
end = time.time()
print("Elapsed (no compilation) = %s" % (end - start))

Now we have compilation out of the way, let's time it more robustly using timeit and compare with a version that doesn't make use of numba.

In [ ]:
numba_time = %timeit -o monte_carlo_pi(N)

In [ ]:
#Plain Python version that doesn't use numba
def python_monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
python_time = %timeit -o python_monte_carlo_pi(N)

In [ ]:
print('Numba version of monte_carlo_pi is {0:.1f} times faster than plain Python'.format(python_time.best/numba_time.best))

## Numba doesn't work on everything

Numba has a lot of functionality but there is still a lot of Python that it does not know about and hence cannot compile to machine code.  The following function is given as an example in the numba documentation as one that Numba can't do anything with

In [ ]:
from numba import jit
import pandas as pd

x = {'a': [1, 2, 3], 'b': [20, 30, 40]}

@jit
def use_pandas(a): 
    df = pd.DataFrame.from_dict(a) 
    df += 1                        
    return df.cov()                

In [ ]:
use_pandas(x)

You might expect an error message when numba is given a function it can't compile but instead everything seemed to have just worked.

To dig deeper, let's create a version of this function without any numba decorations.

In [ ]:
def use_pandas_nonumba(a): 
    df = pd.DataFrame.from_dict(a) 
    df += 1                        
    return df.cov()   

In [ ]:
%timeit use_pandas_nonumba(x)

In [ ]:
%timeit use_pandas(x)

The numba version is actually slightly slower!  Why?

What is happening behind the scenes is that Numba tries to compile the function, realises it doesn't know anything about pandas and so gives up and just runs the function as a normal Python routine.  All of the checking adds a little overhead and we get the opposite of what we were trying to achieve -- slightly slower code instead of much faster code.

You might want to know about Numba's failure. If so, you can use the `nopython` flag which tells Numba that you only want it to produce pure compiled functions -- no interaction with the python interpreter at all.

In [ ]:
@jit(nopython=True)
def use_pandas(a): 
    df = pd.DataFrame.from_dict(a) 
    df += 1                        
    return df.cov()  

In [ ]:
# This will no result in an error rather than a slightly slower version of the Python funcion
use_pandas(x)

## Going Parallel

It is also possible to create parallel code using Numba.  The easiest way to proceed is to add `parallel=True` to the Numba decorator for a function that's already working in `nopython` mode and change `range` to its parallel equivalent `prange`.

In [ ]:
from numba import prange

@jit(parallel=True)
def parallel_monte_carlo_pi(nsamples):
    acc = 0
    for i in prange(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
numba_parallel_time = %timeit -o parallel_monte_carlo_pi(N)

Note that Numba is doing certain things for us automatically to make this work. For example Numba ensures that in the loop `x`, `y` and `acc` are unique to each thread, and then performs a reduction operation at the end of the loop to combine the individual `acc` values into a single value.

I recommend reading the [Numba documentation](https://numba.pydata.org/numba-doc/latest/user/parallel.html) to learn more about the automatic parallelization. In particular:

- Numba attempts to automatically identify operations that can run in parallel, this requires no modifications to the user program.
- When using `prange` the user is required to make sure the loop interations don't have dependencies, except for supported reductions. 

Recall how long this took in serial mode

In [ ]:
numba_serial_time = %timeit -o monte_carlo_pi(N)

and the original, plain Python version

In [ ]:
python_time = %timeit -o python_monte_carlo_pi(N)

In [ ]:
print('The parallel numba version was {0:.1f} times faster than the serial numba version and'.
      format(numba_serial_time.best/numba_parallel_time.best))
print('it was {0:.1f} times faster than the plain python version'.
     format(python_time.best/numba_parallel_time.best))

Numba also automatically ensures that each thread will get a different random number seed but, at the time of writing, I can't figure out how to determine what these seeds are.  

It should also be noted that the underlying Random Number Generator used by **random.random** is [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) which has a very large period.  Although it is likely that multiple random streams created with different seeds won't overlap, it isn't guaranteed 

A discussion on this topic can be found at https://github.com/numba/numba/issues/2486 

## Exercises

1. Produce a parallel Numba version of the `sieve_primes` function.
2. Try to write a parallel Numba version of the Asian option pricing routine below
3. If you have any python code that does something of interest to you, try to write an accelerated version in Numba

In [ ]:
import math # This is the standard Pyton math module. Not the numpy one
import random # This is the standard Python random module.  Not the numpy one

def Asian(so,k,r,v,t,m,n):
    """
    I have not identified what the arguments mean since the original MATLAB code didn't either. 
    This doc-string will be updated once I learn what they are!
    """
    dt = t/m
    AsianPayoffSum = 0
    for i in range(1,n+1):
        s = so
        stSum = so
        at = so
        for j in range(1,m+1):
            st = s * math.exp(((r-v**2/2)*dt) + (v*random.normalvariate(0,1)*math.sqrt(dt)))
            stSum = stSum + st 
            at = stSum/(j+1)
            s = st
        AsianPayoff = max(at-k,0);
        AsianPayoffSum = AsianPayoffSum + AsianPayoff;
    AsianCall = math.exp(-r*t)*(AsianPayoffSum/n)
    return(AsianCall)

## Numba - Next steps

This session has provided just an outline of some of the functionality provided by Numba.  Some of the topics that haven't been covered include

* GPU Support - Numba has strong support for [NVIDIA GPUs](http://numba.pydata.org/numba-doc/latest/cuda/index.html) and some [support for AMD](http://numba.pydata.org/numba-doc/latest/roc/index.html).
* Numba has support for different [threading layers](http://numba.pydata.org/numba-doc/latest/user/threading-layer.html) including OpenMP and Intel Threading Building Blocks (TBB)
* Numba can make use of the [Intel Short Vector Math Library (SVML)](http://numba.pydata.org/numba-doc/latest/user/performance-tips.html#intel-svml) for fast evaluation of trigonometric functions, square roots, exponentials and many more.
* The code compiled by Numba makes use of many of your CPU's features including SIMD instructions.  Detailed diagnostics concerning the compilation can be obtained.